Класс для графа

In [79]:
class Graph:
    def __init__(self, n: int):
        self.n: int = n # number of vertex
        self.adj:list = [[] for i in range(n)] # An array of adjacency lists

    def add_edge(self, v: int, w: int):
        self.adj[w].append(v)
        self.adj[v].append(w)

Функции для подсчета разных степеней

In [80]:
def saturated_degree(graph: Graph, index: int, colors: list): # подсчет степени насыщенности
    used_colors = set()
    for neighbour in graph.adj[index]:
        if colors[neighbour]:
            used_colors.add(colors[neighbour])
    return len(used_colors)

In [81]:
def incidence_degree(graph: Graph, index: int, colors: list): # подсчет степени инцидентности
    inc_degree = 0
    for vertex in graph.adj[index]:
        inc_degree += colors[vertex] != 0
    return inc_degree

In [82]:
def large_degree(graph: Graph, index: int): # подсчет количества соседей
    return len(graph.adj[index])

Раскраска новой вершины и выбор для нее цвета

In [83]:
def color(graph: Graph, index: int, colors: list): # раскраска новой вершины
    used_colors = []
    for neighbour in graph.adj[index]:
        if colors[neighbour]:
            used_colors.append(colors[neighbour])

    for color in range(1, graph.n + 1):
        if color not in used_colors:
            colors[index] = color
            return

Для начала создадим код SDO раскраски (Saturated Degree Ordering)

In [84]:
def SDO(graph: Graph):
    colors = [0] * graph.n
    no_of_colored_nodes = 0
    while no_of_colored_nodes < graph.n:
        max_degree = -1
        index = -1
        for i in range(graph.n):
            if not colors[i]:
                degree = saturated_degree(graph, i, colors)
                if degree > max_degree:
                    max_degree = degree
                    index = i
        color(graph, index, colors)
        no_of_colored_nodes += 1

    return colors

IDO раскраска (Incidence degree ordering)

In [85]:
def IDO(graph: Graph):
    colors = [0] * graph.n
    no_of_colored_nodes = 0
    while no_of_colored_nodes < graph.n:
        max_degree = -1
        index = -1
        for i in range(graph.n):
            if not colors[i]:
                degree = incidence_degree(graph, i, colors)
                if degree > max_degree:
                    max_degree = degree
                    index = i
        color(graph, index, colors)
        no_of_colored_nodes += 1

    return colors

LDO раскраска (Largest degree ordering)

In [86]:
def LDO(graph: Graph):
    colors = [0] * graph.n
    no_of_colored_nodes = 0
    while no_of_colored_nodes < graph.n:
        max_degree = -1
        index = -1
        for i in range(graph.n):
            if not colors[i]:
                degree = large_degree(graph, i)
                if degree > max_degree:
                    max_degree = degree
                    index = i
        color(graph, index, colors)
        no_of_colored_nodes += 1

    return colors

Первый предложенный алгоритм (LDO + IDO)

In [87]:
def first_proposed_algorithm(graph: Graph):
    colors = [0] * graph.n
    no_of_colored_nodes = 0
    while no_of_colored_nodes < graph.n:
        max_degree = -1
        index = -1
        for i in range(graph.n):
            if not colors[i]:
                degree = large_degree(graph, i)
                if degree > max_degree:
                    max_degree = degree
                    index = i
                if degree == max_degree:
                    if incidence_degree(graph, i, colors) > incidence_degree(graph, index, colors):
                        index = i
        color(graph, index, colors)
        no_of_colored_nodes += 1

    return colors

Второй предложенный алгоритм (SDO + LDO)

In [88]:
def second_proposed_algorithm(graph: Graph):
    colors = [0] * graph.n
    no_of_colored_nodes = 0
    while no_of_colored_nodes < graph.n:
        max_degree = -1
        index = -1
        for i in range(graph.n):
            if not colors[i]:
                degree = saturated_degree(graph, i, colors)
                if degree > max_degree:
                    max_degree = degree
                    index = i
                if degree == max_degree:
                    if large_degree(graph, i) > large_degree(graph, index):
                        index = i
        color(graph, index, colors)
        no_of_colored_nodes += 1

    return colors

Создание рандомного графа с заданной плотностью

In [89]:
import random

In [90]:
def make_random_graph(n: int, density: int):
    new_graph = Graph(n)

    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < density/100:
                new_graph.add_edge(i, j)

    return new_graph

Датафрейм для результатов

In [91]:
import pandas as pd

In [92]:
results = pd.DataFrame(columns=['No. of Vertices', 'Density', 'LDO', 'IDO', 'Proposed Algorithm 1', 'SDO', 'Proposed Algorithm 2'])

Повторение результатов исследования об использовании цветов для раскраски

In [93]:
densities = [25, 50, 75]
possible_n = [200, 1000]

for n in possible_n:
    for density in densities:
        graph = make_random_graph(n, density)

        ldo = max(LDO(graph))
        ido = max(IDO(graph))
        first_proposed = max(first_proposed_algorithm(graph))
        sdo = max(SDO(graph))
        second_proposed = max(second_proposed_algorithm(graph))

        results.loc[len(results.index)] = [n, density, ldo, ido, first_proposed, sdo, second_proposed]


In [94]:
results

,No. of Vertices,Density,LDO,IDO,Proposed Algorithm 1,SDO,Proposed Algorithm 2
0,200,25,17,19,17,16,16
1,200,50,34,34,33,30,31
2,200,75,53,58,52,54,51
3,1000,25,62,64,62,58,58
4,1000,50,124,127,123,115,114
5,1000,75,215,216,212,203,201
